## Setup environment

In [ ]:
# pip install langchain langchain-openai pydantic langfuse

In [5]:
# setup langfuse environment variables
import os
from langfuse import Langfuse
from langfuse import get_client
from langfuse.langchain import CallbackHandler

# setup langfuse
LANGFUSE_TRACING_ENABLED = True
Langfuse()
langfuse = get_client()
langfuse_handler = CallbackHandler()

## Define the Pydantic Model for Output Parsing

In [6]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

class MeetingNotes(BaseModel):
    summary: str = Field(description="A two-sentence summary of the meeting's main discussion points.")
    action_items: list[str] = Field(description="A list of action items, each prefaced with a dash ('-').")

## Register and Fetch Prompt Template from Langfuse

In [7]:
def _ensure_prompt(name: str, 
                   prompt: str, 
                   model=os.getenv("MODEL"), 
                   temperature=0.0,
                   type="text",
                   labels = "production",
                   update_existing=False):
    """
    Ensure that a prompt with the given name exists in Langfuse.
    If it does not exist, create it with the specified parameters.
    """
    try:
        langfuse.get_prompt(name)
        if update_existing:
            langfuse.create_prompt(name = name, 
                                   prompt = prompt, 
                                   config={"model": model, "temperature": temperature},
                                   labels=[labels], 
                                   type=type,
                                   )
        else:
            pass
            
    except Exception as e:
        print("exception in _ensure_prompt:", e)
    langfuse.create_prompt(
        name=name,
        prompt=prompt,
        config={"model": model, "temperature": temperature},
        labels=[labels],
        type=type,
    )


def register_prompts(name: str = "analyse-market-sentiment",
                     update_existing=False) -> None:
    """
    Register or update the necessary prompts in Langfuse.
    """
    if name == "analyse-market-sentiment":
        _ensure_prompt(
            name,
            """
            You are a financial analyst. Given recent news headlines, analyze the sentiment and extract the following in JSON format:

            {format_instructions}

            News:
            {news}

            Company Name: {company_name}
            Stock Code: {stock_code}
            Make sure to format your response as JSON using the following schema:\n{format_instructions}
            """,
            update_existing=update_existing)
    elif name == "summarize-meeting":
        _ensure_prompt(
            name,
            """
            You are a meeting assistant.
            1. Summarize the meeting transcript in exactly two sentences.
            2. A list of all action items mentioned, each prefaced with a dash (‘-’).
            Return the result strictly as JSON with keys "summary" and 
            "action_items".
            Transcript:
            {transcript}
            Make sure to format your response as JSON using the following schema:\n{format_instructions}
            """,
            update_existing=update_existing)
    elif name == "summarize-meeting-retry":
        _ensure_prompt(
            name,
            """
            You are a meeting assistant.
            1. Summarize the meeting transcript in exactly two sentences.
            2. A list of all action items mentioned, each prefaced with a dash (‘-’).
            Return the result strictly as JSON with keys "summary" and 
            "action_items".
            Transcript:
            {transcript}
            Make sure to format your response as JSON using the following schema:\n{format_instructions}
            Please output valid JSON only
            """,
            update_existing=update_existing)
    else:
        print("pass correct prompt name")

In [8]:
register_prompts(name="summarize-meeting", update_existing=False)
register_prompts(name="summarize-meeting-retry", update_existing=False)

In [9]:
print(langfuse.get_prompt("summarize-meeting").prompt)
print(langfuse.get_prompt("summarize-meeting-retry").prompt)


            You are a meeting assistant.
            1. Summarize the meeting transcript in exactly two sentences.
            2. A list of all action items mentioned, each prefaced with a dash (‘-’).
            Return the result strictly as JSON with keys "summary" and 
            "action_items".
            Transcript:
            {transcript}
            Make sure to format your response as JSON using the following schema:
{format_instructions}
            

            You are a meeting assistant.
            1. Summarize the meeting transcript in exactly two sentences.
            2. A list of all action items mentioned, each prefaced with a dash (‘-’).
            Return the result strictly as JSON with keys "summary" and 
            "action_items".
            Transcript:
            {transcript}
            Make sure to format your response as JSON using the following schema:
{format_instructions}
            Please output valid JSON only
            


## Set Up the ChatOpenAI Model and Output Parser

In [10]:
from langchain_openai import AzureChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
import json

def transcript_summarizer_chain(transcript: str,
                                langfuse_prompt_name : str = "summarize-meeting") -> dict:
    """
    Extract meeting notes from a transcript using Langfuse and Azure OpenAI.
    
    Args:
        transcript (str): The meeting transcript to analyze.
        
    Returns:
        dict: A dictionary containing the summary and action items.
    """
      
    # call parser
    output_parser = PydanticOutputParser(pydantic_object=MeetingNotes)
    # Call the llm
    llm_call = AzureChatOpenAI(api_key= os.getenv("AZURE_OPENAI_API_KEY"),
            azure_endpoint= os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_version=os.getenv("OPENAI_API_VERSION"),
            model=os.getenv("MODEL")
            )
    # Create a prompt template by fetching prompt from langfuse
    fetch_prompt = langfuse.get_prompt(langfuse_prompt_name)
    filled_prompt = ChatPromptTemplate.from_messages([
        ('system', fetch_prompt.get_langchain_prompt()),
        ("user", "transcript:\n\n{transcript}")
    ]).partial(format_instructions=output_parser.get_format_instructions())

    # Combine the prompt template with the parser and model in a chain
    chain = filled_prompt | llm_call | output_parser
    # Invoke the chain with the transcript
    result = chain.invoke({"transcript": transcript},config={"callbacks":[langfuse_handler]})
    return result

## call transcript_summarizer_chain

In [11]:
def extract_meeting_notes(transcript: str,
                          retry_count: int = 0,
                          max_retries: int = 1) -> dict:
    """
    Extract meeting notes from a transcript by calling function transcript_summarizer_chain.
    
    Args:
        transcript (str): The meeting transcript to analyze.
        max_retries (int): Maximum number of retries if the operation fails (default: 1).
        
    Returns:
        dict: A dictionary containing the summary and action items.
        
    Raises:
        Exception: If the operation fails after all retries are exhausted.
    """
    while retry_count <= max_retries:
        print("trial no", retry_count)
        try:
            result = transcript_summarizer_chain(transcript,
                                                      "summarize-meeting")
            
            result_json = result.model_dump_json(indent=2)
            return json.loads(result_json)
        except json.JSONDecodeError as e:
            print("JSON decoding error:", e)
            retry_count += 1
            print("trial no", retry_count)
            result = transcript_summarizer_chain(transcript,
                                                      "summarize-meeting-retry")

            result_json = result.model_dump_json(indent=2)
            return json.loads(result_json)
        except Exception as e:
            print("Error occured:", e)
            retry_count += 1
            print("trial no", retry_count)
            result = transcript_summarizer_chain(transcript,
                                                      "summarize-meeting")
            result_json = result.model_dump_json(indent=2)
            return json.loads(result_json)
        
    else:
        print(f"Retry Exhausted: {e}")
        raise Exception("Failed to get the the transcript summary after retry.")

## Test sample results

#### Sample 1

In [12]:
transcript = """
Alice: Welcome everyone. Today we need to finalize the Q3 
roadmap.
Bob: I’ve emailed the updated feature list—please review by 
Friday.
Carol: I’ll set up the user‐testing sessions next week.
Dan: Let’s push the new UI mockups to staging on Wednesday.
Alice: Great. Also, can someone compile the stakeholder 
feedback into a slide deck?
Bob: I can handle the slide deck by Monday.
Alice: Thanks, team. Meeting adjourned.
"""

result_json = extract_meeting_notes(transcript = transcript,
                                retry_count = 0,
                                max_retries = 1)

print(result_json)

trial no 0
{'summary': 'The meeting focused on finalizing the Q3 roadmap with a review of an updated feature list and planning user-testing sessions. Action items were assigned including preparing a slide deck from stakeholder feedback and pushing UI mockups to staging.', 'action_items': ['- Review the updated feature list by Friday.', '- Set up user-testing sessions next week.', '- Push the new UI mockups to staging on Wednesday.', '- Compile the stakeholder feedback into a slide deck by Monday.']}


#### Sample 2

In [13]:
transcript = """
Host: Let’s kick off our marketing sync.
Emma: The social campaign draft is 80% done; I’ll share it 
today.
Frank: I spoke with the design team—they’ll deliver assets by 
Tuesday.
Emma: Once we have assets, I’ll schedule the ads for next week.
George: Reminder: submit your budget requests before end of 
day.
Host: Noted. I’ll send out the final budget spreadsheet.
"""

result_json = extract_meeting_notes(transcript = transcript,
                                retry_count = 0,
                                max_retries = 1)

print(result_json)

trial no 0
{'summary': 'The team discussed the progress of the marketing sync, with Emma indicating that the social campaign draft is nearly complete and assets will be delivered by the design team by Tuesday. Additionally, George reminded everyone to submit their budget requests by the end of the day, and the host confirmed they would send out the final budget spreadsheet.', 'action_items': ['- Emma to share the social campaign draft today.', '- Frank to ensure design team delivers assets by Tuesday.', '- Emma to schedule ads for next week after receiving assets.', '- Team to submit budget requests before end of day.', '- Host to send out the final budget spreadsheet.']}


#### Sample 3

In [14]:
transcript = """
John: Sales numbers for April are in—up 12% over target.
Lia: I’ll distribute the detailed report to leadership this 
afternoon.
Mike: We need to train two new reps to handle increased volume.
Lia: I’ll coordinate recruiting with HR by end of week.
John: Excellent. Let’s aim to onboard them by May 15th
"""

result_json = extract_meeting_notes(transcript = transcript,
                                retry_count = 0,
                                max_retries = 1)

print(result_json)

trial no 0
{'summary': 'The meeting discussed the successful sales numbers for April, which were up 12% over target. Action items were established for Lia to distribute a report and coordinate recruiting, aiming to onboard two new representatives by May 15th.', 'action_items': ['- Lia will distribute the detailed report to leadership this afternoon.', '- Lia will coordinate recruiting with HR by the end of the week.', '- Aim to onboard the new reps by May 15th.']}


#### Sample 4

In [15]:
transcript = """
HR: We need to update the travel‐expense policy.
Alex: I’ll draft the new policy doc and share with legal.
HR: Please include guidelines on per diem limits.
Legal: I’ll review and provide feedback within three days.
HR: Once finalized, schedule a companywide announcement.
"""

result_json = extract_meeting_notes(transcript = transcript,
                                retry_count = 0,
                                max_retries = 1)

print(result_json)

trial no 0
{'summary': 'The meeting focused on updating the travel-expense policy, with Alex agreeing to draft the new policy document. Legal will review the draft and provide feedback, and once finalized, an announcement will be scheduled for the entire company.', 'action_items': ['- Alex will draft the new policy document and share it with legal.', '- Include guidelines on per diem limits in the draft.', '- Legal will review the document and provide feedback within three days.', '- Schedule a companywide announcement once the policy is finalized.']}
